In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
import pickle
import random

In [2]:
import json
import os
import json
import random
import logging
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support



def convert_data_to_spacy(JSON_FilePath):
    try:
        training_data = []
        lines = []
        with open(JSON_FilePath, "r", encoding="utf-8") as f:
            lines = f.readlines()

        for line in lines:
            data = json.loads(line)
            text = data["content"]
            entities = []
            for annotation in data["annotation"]:
                # only a single point in text annotation.
                point = annotation["points"][0]
                labels = annotation["label"]
                # handle both list of labels or a single label.
                if not isinstance(labels, list):
                    labels = [labels]

                for label in labels:
                    entities.append((point["start"], point["end"] + 1, label))

            training_data.append((text, {"entities": entities}))

        return training_data
    except Exception as e:
        logging.exception(
            "Unable to process " + JSON_FilePath + "\n" + "error = " + str(e)
        )
        return None


In [3]:
train_data = convert_data_to_spacy("Entity Recognition in Resumes.json")
train_data[0]

('Govardhana K\nSenior Software Engineer\n\nBengaluru, Karnataka, Karnataka - Email me on Indeed: indeed.com/r/Govardhana-K/\nb2de315d95905b68\n\nTotal IT experience 5 Years 6 Months\nCloud Lending Solutions INC 4 Month • Salesforce Developer\nOracle 5 Years 2 Month • Core Java Developer\nLanguages Core Java, Go Lang\nOracle PL-SQL programming,\nSales Force Developer with APEX.\n\nDesignations & Promotions\n\nWilling to relocate: Anywhere\n\nWORK EXPERIENCE\n\nSenior Software Engineer\n\nCloud Lending Solutions -  Bangalore, Karnataka -\n\nJanuary 2018 to Present\n\nPresent\n\nSenior Consultant\n\nOracle -  Bangalore, Karnataka -\n\nNovember 2016 to December 2017\n\nStaff Consultant\n\nOracle -  Bangalore, Karnataka -\n\nJanuary 2014 to October 2016\n\nAssociate Consultant\n\nOracle -  Bangalore, Karnataka -\n\nNovember 2012 to December 2013\n\nEDUCATION\n\nB.E in Computer Science Engineering\n\nAdithya Institute of Technology -  Tamil Nadu\n\nSeptember 2008 to June 2012\n\nhttps://www

In [ ]:
import spacy
from spacy.tokens import Doc, Span
from spacy.util import filter_spans

# Load the model and the dataset
nlp = spacy.load("en_core_web_sm")
 # Your dataset here

# Iterate over the examples in the dataset


C:\Users\hp\.conda\envs\tensor\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [50]:
from spacy.training.example import Example
nlp = spacy.blank('en')
def train_model(train_data):
    # Checking if NER is present in pipeline
    if "ner" not in nlp.pipe_names:
        # creating NER pipe if not present
        ner = nlp.add_pipe("ner")
    else:
        ner = nlp.get_pipe("ner")
    l1=[]
    
    for _, annotation in train_data:
        # Getting each tuple at a time from 'entities' key in dictionary at index[1] i.e.,(0, 15, 'Name') and so on
        for ent in annotation["entities"]:
            # here we are adding only labels of each tuple from entities key dict, eg:- 'Name' label of (0, 15, 'Name')
            #ner.add_label(ent[2])
            l1.append(ent[2])
    l1=list(set(l1))
    print(l1)
    for i in range(len(l1)):
        ner.add_label(l1[i])
   
    for text, annotation in train_data:
        # Getting each tuple at a time from 'entities' key in dictionary at index[1] i.e.,(0, 15, 'Name') and so on
        entities = annotation["entities"]

        doc = nlp.make_doc(text)
        spans = []
        for start, end, label in entities:
            span = doc.char_span(start, end, label=label)
            if span is not None:
                spans.append(span)
                
        
    
   
        filtered_spans = spacy.util.filter_spans(spans)
    
    # Replace the entities in the example with the filtered spans
        annotation["entities"] = [(span.start_char, span.end_char, span.label_) for span in filtered_spans]
    
        
 
        
    other_pipes = [pipe for pipe in nlp.pipe_names if (pipe != "ner")]
    with nlp.select_pipes(disable=other_pipes):
       
        optimizer = nlp.begin_training()
        for itn in range(10):
            print("Starting iteration " + str(itn))
            random.shuffle(train_data)  # shuffling data in every iteration
            losses = {}

            # convert the (text, annotation) tuples to Example objects
            examples = []
            for text, annotation in train_data:
                examples.append(Example.from_dict(nlp.make_doc(text), annotation))
                
            # batch up the examples using spaCy's minibatch
            for batch in spacy.util.minibatch(examples, size=2):
                try:
                    nlp.update(
                        batch,  # batch of Example objects
                        drop=0.2,  # dropout rate - makes it harder to memorize
                        sgd=optimizer,  # callable to update weights
                        losses=losses,  # dictionary to update with the loss, keyed by pipeline component
                    )
                except Exception as e:
                    print(e)
                    pass
            print(losses)
    ruler=nlp.add_pipe('entity_ruler',before='ner')
    skills='abc.jsonl'
    ruler.from_disk(skills)
    patterns=[
    
    {
        'label':'EMAIL','pattern':[{"TEXT":{"REGEX":"([^@|\s]+@[^@]+\.[^@|\s]+)"}}]
        
    },
    {
        'label':'MOBILE','pattern':[{"TEXT":{"REGEX":"\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4}"}}]
        
    },
     {
        'label':'Name','pattern':[{"TEXT":{"REGEX":"/^[a-zA-Z ]*$/"}}]
        
    }
    
    
    ]
   
    ruler.add_patterns(patterns)    
            
train_model(train_data)

['Name', 'Years of Experience', 'UNKNOWN', 'Skills', 'Companies worked at', 'Designation', 'Email Address', 'Location', 'Graduation Year', 'Degree', 'College Name']
Starting iteration 0
[E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help
[E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're trainin

[E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. You can also use the `debug data` command to validate your JSON-formatted training data. For details, run:
python -m spacy debug data --help
{'ner': 2932.4235680674974}
Starting iteration 6
[E024] Could not find an optimal move to supervise the parser. Usually, this means that the model can't be updated in a way that's valid and satisfies the correct annotations specified in the GoldParse. For example, are all labels added to the model? If you're training a named entity recognizer, also make sure that none of your annotated entity spans have leading or trailing whitespace or punctuation. 

In [51]:
m1=nlp("Java")
from spacy import displacy
doc = nlp(train_data[0][0])
displacy.render(doc, style="ent", jupyter=True)


In [54]:
dict_data={}
for ent in doc.ents:
    if ent.label_.upper() not in dict_data:
        dict_data[ent.label_.upper()]=[ent.text]
    else:
        dict_data[ent.label_.upper()].append(ent.text)
    #print(f"{ent.label_.upper():{30}}-{ent.text}")

NAME                          -Kavya U.
COMPANIES WORKED AT           -Accenture
LOCATION                      -Bengaluru
MOBILE                        -indeed.com/r/Kavya-U/049577580b3814e6
COMPANIES WORKED AT           -Accenture
SKILL                         -Wireless
SKILL                         -Engineering
SKILL                         -Access Network
SKILLS                        -Verilog HDL
• Worked on
SKILL                         -Linux
SKILL                         -Simulation
SKILL                         -documentation
GRADUATION YEAR               -2007
SKILL                         -Design
MOBILE                        -https://www.indeed.com/r/Kavya-U/049577580b3814e6?isid=rex-download&ikw=download-top&co=IN
COLLEGE NAME                  -Manipal Academy of Higher Education
SKILL                         -Engineering
SKILL                         -Engineering
COLLEGE NAME                  -Srinivas Institute of Technology
COLLEGE NAME                  -Visvesvaraya Tec

In [55]:
print(dict_data)

{'NAME': ['Kavya U.'], 'COMPANIES WORKED AT': ['Accenture', 'Accenture'], 'LOCATION': ['Bengaluru'], 'MOBILE': ['indeed.com/r/Kavya-U/049577580b3814e6', 'https://www.indeed.com/r/Kavya-U/049577580b3814e6?isid=rex-download&ikw=download-top&co=IN'], 'SKILL': ['Wireless', 'Engineering', 'Access Network', 'Linux', 'Simulation', 'documentation', 'Design', 'Engineering', 'Engineering'], 'SKILLS': ['Verilog HDL\n• Worked on', 'coding (Less than 1 year), HDL (Less than 1 year),', 'Verilog HDL\n• Knowledge of RTL coding, FSM based designs.\n• Understanding of UART, AMBA protocol\n• Platforms:'], 'GRADUATION YEAR': ['2007'], 'COLLEGE NAME': ['Manipal Academy of Higher Education', 'Srinivas Institute of Technology', 'Visvesvaraya Technological University', 'Vidyodaya P.U. College']}
